In [ ]:
torch.cuda.empty_cache()

NameError: name 'torch' is not defined

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=31fb75c675d77bc342ae9362a31263a7efab407a2478b444820c510e6065e986
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from torch import nn
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from transformers import (
    ViTModel,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    AutoProcessor,
    AutoModelForVision2Seq,
    BertTokenizer
)
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('wordnet')
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from rouge_score import rouge_scorer
import random
import json

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Deep_Learning_Assignment/custom_captions_dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: custom_captions_dataset/train/train_3367.jpg  
  inflating: custom_captions_dataset/train/train_3368.jpg  
  inflating: custom_captions_dataset/train/train_3370.jpg  
  inflating: custom_captions_dataset/train/train_3372.jpg  
  inflating: custom_captions_dataset/train/train_3373.jpg  
  inflating: custom_captions_dataset/train/train_3383.jpg  
  inflating: custom_captions_dataset/train/train_3392.jpg  
  inflating: custom_captions_dataset/train/train_3412.jpg  
  inflating: custom_captions_dataset/train/train_3413.jpg  
  inflating: custom_captions_dataset/train/train_3417.jpg  
  inflating: custom_captions_dataset/train/train_3422.jpg  
  inflating: custom_captions_dataset/train/train_3431.jpg  
  inflating: custom_captions_dataset/train/train_3432.jpg  
  inflating: custom_captions_dataset/train/train_3435.jpg  
  inflating: custom_captions_dataset/train/train_3442.jpg  
  inflating: custom_captions_dataset/train/train_

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
class ImageCaptionDataset(Dataset):
    def __init__(self, data_path, image_folder, transform=None, max_length=128):
        """
        Args:
            data_path (str): Path to the CSV file with annotations
            image_folder (str): Directory with all the images
            transform (callable, optional): Optional transform to be applied on an image
            max_length (int): Maximum length of the caption
        """
        self.data = pd.read_csv(data_path)
        self.image_folder = image_folder
        self.transform = transform
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        filename = self.data.iloc[idx]['filename']  # Get filename from CSV
        img_name = os.path.join(self.image_folder, filename)
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        caption = self.data.iloc[idx]['caption']  # Using 'caption' column

        # Tokenize caption
        caption_encoding = self.tokenizer(
            caption,
            padding='max_length',
            max_length=self.max_length,
            truncation=True,
            return_tensors='pt'
        )

        return {
            'image': image,
            'input_ids': caption_encoding['input_ids'].squeeze(),
            'attention_mask': caption_encoding['attention_mask'].squeeze(),
            'caption': caption,
            'filename': filename
        }



In [ ]:
def zero_shot_captioning(image_path, model_name="smolvlm"):
    try:
        if model_name == "smolvlm":
            processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-500M-Instruct")
            model = AutoModelForVision2Seq.from_pretrained(
                "HuggingFaceTB/SmolVLM-500M-Instruct",
                torch_dtype=torch.float16,
                _attn_implementation="eager"
            ).to("cuda" if torch.cuda.is_available() else "cpu")

            # 1. Prepare image and messages with proper structure
            image = Image.open(image_path).convert("RGB").resize((384, 384))
            messages = [{
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": "Describe this image:"}
                ]
            }]

            # 2. Apply chat template
            prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

            # 3. Process with both image and formatted prompt
            inputs = processor(
                text=prompt,
                images=image,
                return_tensors="pt"
            ).to(model.device)

            # 4. Generate with proper parameters
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=50,
                do_sample=False
            )

            caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
            return caption

        else:
            raise ValueError(f"Model {model_name} not supported.")

    except Exception as e:
        print(f"Error: {e}")
        return "Error generating caption"


In [ ]:
from transformers import ViTModel


class ImageCaptionModel(nn.Module):
    def __init__(self, vit_model_name='google/vit-base-patch16-224-in21k', gpt_model_name='gpt2', projection_dim=768):
        """
        Custom Encoder-Decoder Model for Image Captioning using ViT as an encoder

        Args:
            vit_model_name (str): Name of the ViT model to use as encoder
            gpt_model_name (str): Name of the GPT model to use as decoder
            projection_dim (int): Dimension of the projection layer
        """
        super(ImageCaptionModel, self).__init__()

        # Load pre-trained ViT model as the encoder (using ViT-Small as specified in assignment)
        self.encoder = ViTModel.from_pretrained(vit_model_name, token = "xxxxxx")
        encoder_dim = self.encoder.config.hidden_size

        # Load pre-trained GPT-2 model as the decoder
        self.decoder = GPT2LMHeadModel.from_pretrained(gpt_model_name)
        decoder_dim = self.decoder.config.n_embd

        # Projection layer to map encoder features to decoder dimension
        self.projection = nn.Linear(encoder_dim, projection_dim)

        # Cross-attention layers for decoder to attend to image features
        self.cross_attn = nn.MultiheadAttention(projection_dim, num_heads=8, batch_first=True)

        # Map projected image features to decoder dimension for conditioning
        self.img_to_decoder_dim = nn.Linear(projection_dim, decoder_dim)

        # Freeze some of the encoder layers to prevent overfitting and reduce memory usage
        for param in list(self.encoder.parameters())[:6]:
            param.requires_grad = False

    def forward(self, pixel_values, input_ids, attention_mask=None):
        """
        Forward pass of the model

        Args:
            pixel_values (torch.Tensor): Batch of images
            input_ids (torch.Tensor): Tokenized captions
            attention_mask (torch.Tensor, optional): Attention mask for captions

        Returns:
            torch.Tensor: Logits for predicting the next token
        """
        # Encode the image
        encoder_outputs = self.encoder(pixel_values)
        image_features = encoder_outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]

        # Project image features to the common dimension
        projected_features = self.projection(image_features)

        # Process through GPT-2
        # First, conditioning with image features
        img_features_decoder_dim = self.img_to_decoder_dim(projected_features[:, 0].unsqueeze(1))

        # Get embeddings from GPT-2
        decoder_outputs = self.decoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True
        )

        # Get hidden states from the last layer
        hidden_states = decoder_outputs.hidden_states[-1]

        # Project hidden states to the common dimension for cross-attention
        projected_hidden_states = F.linear(hidden_states, self.projection.weight.t())

        # Apply cross-attention between text features and image features
        cross_attended, _ = self.cross_attn(projected_hidden_states, projected_features, projected_features)

        # Combine with original hidden states (add residuals)
        final_hidden_states = hidden_states + cross_attended

        # Project to vocabulary through decoder's language modeling head
        lm_logits = self.decoder.lm_head(final_hidden_states)

        return lm_logits

In [ ]:
def train_model(model, dataloader, optimizer, criterion, device, epochs):
    """
    Train the encoder-decoder model.

    Args:
        model (nn.Module): Custom image captioning model.
        dataloader (DataLoader): Training data loader.
        optimizer: Optimizer (e.g., Adam).
        criterion (Loss): Loss function.
        device (str): Device to use ('cuda' or 'cpu').
        epochs (int): Number of epochs.
    """
    model.train()
    print("Starting training...")

    for epoch in range(epochs):
        epoch_loss = 0
        batch_count = 0

        for batch in dataloader:
            batch_count += 1

            # Move batch data to device
            images = batch['image'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Forward pass
            outputs = model(
                pixel_values=images,
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            # Calculate loss (shift prediction and targets)
            # We need to shift because we want to predict the next token
            shift_logits = outputs[..., :-1, :].contiguous()
            shift_labels = input_ids[..., 1:].contiguous()

            # Calculate loss
            loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()

            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            epoch_loss += loss.item()

            # Print progress
            if batch_count % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Batch {batch_count}/{len(dataloader)}, Loss: {loss.item():.4f}")

        avg_epoch_loss = epoch_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs} completed. Average Loss: {avg_epoch_loss:.4f}")

        # Save checkpoint after each epoch
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_epoch_loss,
        }, f'model_checkpoint_epoch_{epoch+1}.pth')


In [ ]:
def generate_caption(model, image_tensor, tokenizer, device, max_length=50):
    """
    Generate a caption for a given image using the trained model.

    Args:
        model (nn.Module): Trained image captioning model
        image_tensor (torch.Tensor): Image tensor to generate caption for
        tokenizer: Tokenizer for text processing
        device (str): Device to use ('cuda' or 'cpu')
        max_length (int): Maximum length of generated caption

    Returns:
        str: Generated caption
    """
    model.eval()
    with torch.no_grad():
        # Start with BOS token
        input_ids = torch.tensor([[tokenizer.bos_token_id]]).to(device)

        for _ in range(max_length):
            # Forward pass
            outputs = model(
                pixel_values=image_tensor.unsqueeze(0),
                input_ids=input_ids
            )

            # Get the predicted token
            next_token_logits = outputs[:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)

            # Append the predicted token to input_ids
            input_ids = torch.cat([input_ids, next_token], dim=1)

            # Break if we predict the end of sequence token
            if next_token.item() == tokenizer.eos_token_id:
                break

        # Decode the generated caption
        caption = tokenizer.decode(input_ids[0], skip_special_tokens=True)

    return caption


In [ ]:
def evaluate_model(model, dataloader, device):
    """
    Evaluate model performance using BLEU, ROUGE-L, METEOR.

    Args:
        model (nn.Module): Trained model.
        dataloader (DataLoader): Test data loader.
        device (str): 'cuda' or 'cpu'.

    Returns:
        dict: BLEU, ROUGE-L, METEOR scores for the test set.
    """
    model.eval()
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token

    references = []
    hypotheses = []

    print("Starting evaluation...")

    with torch.no_grad():
        for batch in dataloader:
            # Get images and move to device
            images = batch['image'].to(device)

            # Get ground truth captions
            captions = batch['caption']

            # Generate captions using the model
            for i in range(len(images)):
                image = images[i]
                reference = captions[i]

                # Generate caption using the model
                hypothesis = generate_caption(model, image, tokenizer, device)

                # Process reference and hypothesis for evaluation
                references.append([reference.split()])
                hypotheses.append(hypothesis.split())

    # Calculate BLEU score
    smoothing = SmoothingFunction().method1
    bleu_score = corpus_bleu(references, hypotheses, smoothing_function=smoothing)

    # Calculate ROUGE-L score
    rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_scores = []
    for ref, hyp in zip(references, hypotheses):
        ref_text = ' '.join(ref[0])
        hyp_text = ' '.join(hyp)
        rouge_scores.append(rouge.score(ref_text, hyp_text)['rougeL'].fmeasure)
    rouge_l_score = sum(rouge_scores) / len(rouge_scores)

    # Calculate METEOR score
    meteor_scores = []
    for ref, hyp in zip(references, hypotheses):
        meteor_scores.append(meteor_score([ref[0]], hyp))
    meteor_score_avg = sum(meteor_scores) / len(meteor_scores)

    results = {
        'BLEU': bleu_score,
        'ROUGE-L': rouge_l_score,
        'METEOR': meteor_score_avg
    }

    print(f"Evaluation Results:")
    print(f"BLEU: {bleu_score:.4f}")
    print(f"ROUGE-L: {rouge_l_score:.4f}")
    print(f"METEOR: {meteor_score_avg:.4f}")

    return results

In [ ]:
def evaluate_zero_shot_smolvlm(test_dataloader, test_image_folder):
    """
    Evaluate the SmolVLM model on the test set in zero-shot setting
    """
    references = []
    hypotheses = []

    print("Starting SmolVLM zero-shot evaluation...")

    for batch in test_dataloader:
        filenames = batch['filename']
        captions = batch['caption']

        for filename, ref_caption in zip(filenames, captions):
            img_path = os.path.join(test_image_folder, filename)

            # Generate caption using SmolVLM
            hyp_caption = zero_shot_captioning(img_path, model_name="smolvlm")

            references.append([ref_caption.split()])
            hypotheses.append(hyp_caption.split())

            # Print sample results occasionally
            if len(hypotheses) % 10 == 0:
                print(f"Sample {len(hypotheses)}:")
                print(f"Image: {filename}")
                print(f"Reference: {ref_caption}")
                print(f"Generated: {hyp_caption}")
                print("-" * 50)

    # Calculate metrics
    smoothing = SmoothingFunction().method1
    bleu_score = corpus_bleu(references, hypotheses, smoothing_function=smoothing)

    rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_scores = []
    for ref, hyp in zip(references, hypotheses):
        ref_text = ' '.join(ref[0])
        hyp_text = ' '.join(hyp)
        rouge_scores.append(rouge.score(ref_text, hyp_text)['rougeL'].fmeasure)
    rouge_l_score = sum(rouge_scores) / len(rouge_scores)

    meteor_scores = []
    for ref, hyp in zip(references, hypotheses):
        meteor_scores.append(meteor_score([ref[0]], hyp))
    meteor_score_avg = sum(meteor_scores) / len(meteor_scores)

    results = {
        'BLEU': bleu_score,
        'ROUGE-L': rouge_l_score,
        'METEOR': meteor_score_avg
    }

    print(f"SmolVLM Zero-shot Evaluation Results:")
    print(f"BLEU: {bleu_score:.4f}")
    print(f"ROUGE-L: {rouge_l_score:.4f}")
    print(f"METEOR: {meteor_score_avg:.4f}")

    return results

In [ ]:
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define paths
train_data_path = '/content/custom_captions_dataset/train.csv'
val_data_path = '/content/custom_captions_dataset/val.csv'
test_data_path = '/content/custom_captions_dataset/test.csv'

train_image_folder = '/content/custom_captions_dataset/train'
val_image_folder = '/content/custom_captions_dataset/val'
test_image_folder = '/content/custom_captions_dataset/test'

# Create datasets
train_dataset = ImageCaptionDataset(train_data_path, train_image_folder, transform)
val_dataset = ImageCaptionDataset(val_data_path, val_image_folder, transform)
test_dataset = ImageCaptionDataset(test_data_path, test_image_folder, transform)

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
smolvlm_results = evaluate_zero_shot_smolvlm(test_dataloader, test_image_folder)

Starting SmolVLM zero-shot evaluation...


processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/429 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.2k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

Sample 10:
Image: test_10.jpg
Reference: A painted mannequin head sits on a table. It is a feminine head with arched eyebrows a small nose, big eyes and it is wearing a black top hat and a brown wig. To the right of the head is a metal sculpture clock with a white face.
Generated: User:




Describe this image:
Assistant: The image depicts a mannequin head with a black hat on its head. The mannequin head is positioned to the left side of the image, and it appears to be facing towards the right. The mannequin head has a neutral expression
--------------------------------------------------
Sample 20:
Image: test_20.jpg
Reference: a flood is occuring. there is a lot of water on the road. the hydrant on the sidewalk is red. the garage doors are red in color . there are some pieces of grass growing.
Generated: User:




Describe this image:
Assistant: The image depicts a scene on a concrete surface, likely a parking lot or a street. The central focus is a red fire hydrant, which is prominen

In [ ]:
custom_model = ImageCaptionModel().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=-100)
optimizer = torch.optim.AdamW(custom_model.parameters(), lr=5e-5)

# Train the model
train_model(
    model=custom_model,
    dataloader=train_dataloader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    epochs=5  # Adjust as needed
)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Starting training...
Epoch 1/5, Batch 10/358, Loss: 1.6206
Epoch 1/5, Batch 20/358, Loss: 1.6518
Epoch 1/5, Batch 30/358, Loss: 1.5344
Epoch 1/5, Batch 40/358, Loss: 1.8960
Epoch 1/5, Batch 50/358, Loss: 1.1616
Epoch 1/5, Batch 60/358, Loss: 1.5287
Epoch 1/5, Batch 70/358, Loss: 1.3930
Epoch 1/5, Batch 80/358, Loss: 1.6470
Epoch 1/5, Batch 90/358, Loss: 1.4633
Epoch 1/5, Batch 100/358, Loss: 1.5199
Epoch 1/5, Batch 110/358, Loss: 1.7809
Epoch 1/5, Batch 120/358, Loss: 1.3238
Epoch 1/5, Batch 130/358, Loss: 1.4490
Epoch 1/5, Batch 140/358, Loss: 1.4550
Epoch 1/5, Batch 150/358, Loss: 1.3742
Epoch 1/5, Batch 160/358, Loss: 1.3406
Epoch 1/5, Batch 170/358, Loss: 1.1288
Epoch 1/5, Batch 180/358, Loss: 1.2598
Epoch 1/5, Batch 190/358, Loss: 1.1969
Epoch 1/5, Batch 200/358, Loss: 1.4824
Epoch 1/5, Batch 210/358, Loss: 1.2646
Epoch 1/5, Batch 220/358, Loss: 1.2127
Epoch 1/5, Batch 230/358, Loss: 1.2408
Epoch 1/5, Batch 240/358, Loss: 1.2259
Epoch 1/5, Batch 250/358, Loss: 1.3578
Epoch 1/5, Ba

In [ ]:
custom_model_results = evaluate_model(custom_model, test_dataloader, device)

# Compare results
print("\nPerformance Comparison:")
print("Model\t\tBLEU\t\tROUGE-L\t\tMETEOR")
print(f"SmolVLM\t\t{smolvlm_results['BLEU']:.4f}\t\t{smolvlm_results['ROUGE-L']:.4f}\t\t{smolvlm_results['METEOR']:.4f}")
print(f"Custom\t\t{custom_model_results['BLEU']:.4f}\t\t{custom_model_results['ROUGE-L']:.4f}\t\t{custom_model_results['METEOR']:.4f}")

# Save the final model
torch.save(custom_model.state_dict(), 'image_caption_model_final.pth')

Starting evaluation...
Evaluation Results:
BLEU: 0.0444
ROUGE-L: 0.2836
METEOR: 0.2082

Performance Comparison:
Model		BLEU		ROUGE-L		METEOR
SmolVLM		0.0275		0.2244		0.1747
Custom		0.0444		0.2836		0.2082
